<a href="https://colab.research.google.com/github/akash-soni/NLP-notebooks/blob/main/HTML_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment -

Use DVC or MLFLOW to convert this into a
  1. Create an API which will take the first input and it will genreate next sentences or
  2. Create an UI for this app

# Text generation with RNN

In [ ]:
import os
ROOT = "/content/drive/MyDrive/FDFS_feb_Batch/NLP/1 Text-generation"
os.chdir(ROOT)

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time


In [ ]:
os.makedirs("data", exist_ok=True)

In [ ]:
# samething can be done using config.yaml
class Config:
  path_to_file = os.path.join("data", "html_data.txt")
  seq_length = 100

  Batch_size = 64
  """
  # it will keep 10000 characters of dataset in the memory(RAM),
  so while training data will be randomly picked from buffer
  and the empty space will be filled up by the data from the disk(HDD).
  Buffer is used to increase speed.
  """
  Buffer_size =10000

  """
  Letter embedding
  embedding vector for every character
  ex : for letter 'a' there is 256 size vector
       for letter 'b' there is 256 size vector
       so on...
  """
  embedding_dim = 256

  """
  From embedding layer -----> 1024 GRU Units ----> dense layer of 65(we have 65 characters)
  """
  rnn_units = 1024
  EPOCHS = 30

  checkpoint_dir = "./training_ckpt"


In [ ]:
Config.EPOCHS

30

In [ ]:
Config.path_to_file

'data/html_data.txt'

In [ ]:
text = open(Config.path_to_file, "rb").read().decode(encoding='utf-8')
text[:200]

'<!DOCTYPE html>\r\n<html class="client-nojs" lang="en" dir="ltr">\r\n<head>\r\n<meta charset="UTF-8"/>\r\n<title>Rajasthan - Wikipedia</title>\r\n<script>document.documentElement.className="client-js";RLCONF={"'

In [ ]:
with open(Config.path_to_file) as f:
  t = f.read()


In [ ]:
t[:100]

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title'

In [ ]:
sorted(set(text))

['\t',
 '\n',
 '\r',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '^',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '®',
 '°',
 '±',
 '·',
 'Í',
 'à',
 'á',
 'â',
 'ã',
 'å',
 'ç',
 'è',
 'ê',
 'ì',
 'ñ',
 'õ',
 'ù',
 'ú',
 'ü',
 'ā',
 'ă',
 'Č',
 'č',
 'ĕ',
 'ĝ',
 'š',
 'ų',
 'ź',
 'ż',
 'ž',
 'ə',
 'ʒ',
 'ʰ',
 'ʻ',
 'ˈ',
 'ː',
 '̄',
 '̤',
 'Ε',
 'Ρ',
 'ά',
 'α',
 'ζ',
 'η',
 'ι',
 'κ',
 'λ',
 'ν',
 'σ',
 'τ',
 'Б',
 'И',
 'М',
 'Н',
 'О',
 'Р',
 'С',
 'Т',
 'У',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',


# Converting text to integer vector

In [ ]:
#get all the unique characters in sorted order
vocab = sorted(set(text))
print("no. of uniqe characters are ", len(vocab))

no. of uniqe characters are  507


In [ ]:
#enumerate all the characters to integers
char2idx = {uniChar : idx for idx, uniChar in enumerate(vocab)}

In [ ]:
char2idx

{'\t': 0,
 '\n': 1,
 '\r': 2,
 ' ': 3,
 '!': 4,
 '"': 5,
 '#': 6,
 '$': 7,
 '%': 8,
 '&': 9,
 "'": 10,
 '(': 11,
 ')': 12,
 '*': 13,
 '+': 14,
 ',': 15,
 '-': 16,
 '.': 17,
 '/': 18,
 '0': 19,
 '1': 20,
 '2': 21,
 '3': 22,
 '4': 23,
 '5': 24,
 '6': 25,
 '7': 26,
 '8': 27,
 '9': 28,
 ':': 29,
 ';': 30,
 '<': 31,
 '=': 32,
 '>': 33,
 '?': 34,
 '@': 35,
 'A': 36,
 'B': 37,
 'C': 38,
 'D': 39,
 'E': 40,
 'F': 41,
 'G': 42,
 'H': 43,
 'I': 44,
 'J': 45,
 'K': 46,
 'L': 47,
 'M': 48,
 'N': 49,
 'O': 50,
 'P': 51,
 'Q': 52,
 'R': 53,
 'S': 54,
 'T': 55,
 'U': 56,
 'V': 57,
 'W': 58,
 'X': 59,
 'Y': 60,
 'Z': 61,
 '[': 62,
 '\\': 63,
 ']': 64,
 '^': 65,
 '_': 66,
 'a': 67,
 'b': 68,
 'c': 69,
 'd': 70,
 'e': 71,
 'f': 72,
 'g': 73,
 'h': 74,
 'i': 75,
 'j': 76,
 'k': 77,
 'l': 78,
 'm': 79,
 'n': 80,
 'o': 81,
 'p': 82,
 'q': 83,
 'r': 84,
 's': 85,
 't': 86,
 'u': 87,
 'v': 88,
 'w': 89,
 'x': 90,
 'y': 91,
 'z': 92,
 '{': 93,
 '|': 94,
 '}': 95,
 '®': 96,
 '°': 97,
 '±': 98,
 '·': 99,
 'Í': 

In [ ]:
text

'<!DOCTYPE html>\r\n<html class="client-nojs" lang="en" dir="ltr">\r\n<head>\r\n<meta charset="UTF-8"/>\r\n<title>Rajasthan - Wikipedia</title>\r\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"ee5a7fbc-26bd-4c11-a2b9-75b04f61cb21","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Rajasthan","wgTitle":"Rajasthan","wgCurRevisionId":1095329799,"wgRevisionId":1095329799,"wgArticleId":26291,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Harv and Sfn no-target errors","CS1 Hindi-language sources (hi)","Articles with short description","Short description is different from Wikidata","Wiki

In [ ]:
# creating word text vector
text_as_int = np.array([char2idx[c] for c in text])

text_as_int # all the text is now represented as integer

array([31,  4, 39, ..., 79, 78, 33])

In [ ]:
len(text_as_int)

433039

In [ ]:
text[:13], text_as_int[:13]

('<!DOCTYPE htm', array([31,  4, 39, 50, 38, 55, 60, 51, 40,  3, 74, 86, 79]))

# Get the max sequence length

In [ ]:
print("total length is : ", len(text))

total length is :  433039


In [ ]:
examples_per_epoch = len(text)//(Config.seq_length + 1)
examples_per_epoch

4287

In [ ]:
idx2char = {val:key for key, val in char2idx.items()}

In [ ]:
#slicing each converted character of dataset to integer
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
  print(idx2char[i.numpy()])

<
!
D
O
C
T
Y
P
E
 


In [ ]:
# combining each tensors according to the sequence length
#  Total characters  : 1115394
#  Batch size : 101
#  Total Batches : 1115394//101 = 11043
sequences = char_dataset.batch(Config.seq_length+ 1, drop_remainder = True)
# take
for item in sequences.take(5):
  print("\nSEQUENCE")
  print(repr("".join([idx2char[c] for c in item.numpy()])))


SEQUENCE
'<!DOCTYPE html>\r\n<html class="client-nojs" lang="en" dir="ltr">\r\n<head>\r\n<meta charset="UTF-8"/>\r\n<ti'

SEQUENCE
'tle>Rajasthan - Wikipedia</title>\r\n<script>document.documentElement.className="client-js";RLCONF={"wg'

SEQUENCE
'BreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDate'

SEQUENCE
'Format":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","S'

SEQUENCE
'eptember","October","November","December"],"wgRequestId":"ee5a7fbc-26bd-4c11-a2b9-75b04f61cb21","wgCS'


# creating a target value

Ex : First Citize -> irst Citizen

In [ ]:
# creating input and output chunks for each individual sequence
from numpy.lib.shape_base import split

def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset =  sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
  print("INPUT DATA:\n")
  print(repr("".join([idx2char[i] for i in input_example.numpy()])))

  print("\n\nTARGET_DATA:\n")
  print(repr("".join([idx2char[i] for i in target_example.numpy()])))

INPUT DATA:

'<!DOCTYPE html>\r\n<html class="client-nojs" lang="en" dir="ltr">\r\n<head>\r\n<meta charset="UTF-8"/>\r\n<t'


TARGET_DATA:

'!DOCTYPE html>\r\n<html class="client-nojs" lang="en" dir="ltr">\r\n<head>\r\n<meta charset="UTF-8"/>\r\n<ti'


# Creating training Batches

In [ ]:
dataset =  dataset.shuffle(Config.Batch_size).batch(Config.Batch_size, drop_remainder=True)
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
vocab

['\t',
 '\n',
 '\r',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '^',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '®',
 '°',
 '±',
 '·',
 'Í',
 'à',
 'á',
 'â',
 'ã',
 'å',
 'ç',
 'è',
 'ê',
 'ì',
 'ñ',
 'õ',
 'ù',
 'ú',
 'ü',
 'ā',
 'ă',
 'Č',
 'č',
 'ĕ',
 'ĝ',
 'š',
 'ų',
 'ź',
 'ż',
 'ž',
 'ə',
 'ʒ',
 'ʰ',
 'ʻ',
 'ˈ',
 'ː',
 '̄',
 '̤',
 'Ε',
 'Ρ',
 'ά',
 'α',
 'ζ',
 'η',
 'ι',
 'κ',
 'λ',
 'ν',
 'σ',
 'τ',
 'Б',
 'И',
 'М',
 'Н',
 'О',
 'Р',
 'С',
 'Т',
 'У',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',


In [ ]:
# finidng no. of output classes
Config.vocab_size = len(vocab)
Config.vocab_size

507

# Building a Model

In [ ]:
def build_model(
    vocab_size ,embedding_dim, rnn_units, batch_size
    ):
    model = tf.keras.Sequential([
                                 # vocab_size is input of 65 characters
                                 # embedding_dim is output vector of 256 dimension
                                 # batch_input_shape is batches of 64 will be passed
                                 tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape = [batch_size, None]),
                                 tf.keras.layers.GRU(rnn_units,
                                                     return_sequences = True,
                                                     stateful = True,
                                                     recurrent_initializer = 'glorot_uniform'
                                                     ),
                                 tf.keras.layers.Dense(vocab_size)
                                 ])
    return model

In [ ]:
model = build_model(
    vocab_size= Config.vocab_size,
    embedding_dim=Config.embedding_dim,
    rnn_units = Config.rnn_units,
    batch_size = Config.Batch_size
)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           129792    
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 507)           519675    
                                                                 
Total params: 4,587,771
Trainable params: 4,587,771
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# defining loss
# logits : means we are taking raw output as there is no activation function like a softmax layer
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
checkpoint_prefix = os.path.join(Config.checkpoint_dir, "chkpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True
)

# Compiling model

In [ ]:
history = model.fit(dataset, epochs = Config.EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
66/66 [==============================] - 18s 156ms/step - loss: 3.6717
Epoch 2/30
66/66 [==============================] - 6s 93ms/step - loss: 2.8196
Epoch 3/30
66/66 [==============================] - 6s 97ms/step - loss: 2.4009
Epoch 4/30
66/66 [==============================] - 6s 97ms/step - loss: 2.0602
Epoch 5/30
66/66 [==============================] - 6s 93ms/step - loss: 1.7690
Epoch 6/30
66/66 [==============================] - 6s 94ms/step - loss: 1.5321
Epoch 7/30
66/66 [==============================] - 6s 94ms/step - loss: 1.3505
Epoch 8/30
66/66 [==============================] - 6s 97ms/step - loss: 1.2107
Epoch 9/30
66/66 [==============================] - 6s 94ms/step - loss: 1.0976
Epoch 10/30
66/66 [==============================] - 6s 96ms/step - loss: 1.0025
Epoch 11/30
66/66 [==============================] - 5s 75ms/step - loss: 0.9147
Epoch 12/30
66/66 [==============================] - 6s 91ms/step - loss: 0.8420
Epoch 13/30
66/66 [================

# Loading the model from checkpoint

In [ ]:
# Restoring checkpoint -

tf.train.latest_checkpoint(Config.checkpoint_dir)

'./training_ckpt/chkpt_30'

In [ ]:
# building a model from restored checkpoint
model_from_ckpt = build_model(
    vocab_size= Config.vocab_size,
    embedding_dim=Config.embedding_dim,
    rnn_units = Config.rnn_units,
    batch_size = 1
)

In [ ]:
model_from_ckpt.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            129792    
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 507)            519675    
                                                                 
Total params: 4,587,771
Trainable params: 4,587,771
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# loading the model weights
model_from_ckpt.load_weights(tf.train.latest_checkpoint(Config.checkpoint_dir))

model_from_ckpt.build(tf.TensorShape([1,None]))

In [ ]:
model_from_ckpt.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            129792    
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 507)            519675    
                                                                 
Total params: 4,587,771
Trainable params: 4,587,771
Non-trainable params: 0
_________________________________________________________________


# Prediction

In [ ]:
# prediction function
def generate_text(model, start_string, no_of_chars_to_gen = 1000):
  # convert the input text to nos.
  input_val = [char2idx[s] for s in start_string] # text converted to int
  input_val = tf.expand_dims(input_val, 0) # [x] -> [1, x]

  text_generated = list()

  temperature = 1.0 # normalization factor for output

  model.reset_states() # Resetting the previous states if any while predictions.
  for i in range(no_of_chars_to_gen):
    predictions = model(input_val)

    predictions = tf.squeeze(predictions, 0)

    predictions = predictions / temperature

    print(predictions)
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
    print(predicted_id)

    input_val = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return start_string + "".join(text_generated)


In [ ]:
generate_text(model = model_from_ckpt, start_string="ROMEO", no_of_chars_to_gen=1000)

Streaming output truncated to the last 5000 lines.
   2.4831808   2.7739604  14.825532    3.4447794  -5.964407   -3.8198571
  -0.8850757  -1.4071232  -1.5257107   0.7763674   1.4912635  -4.4794345
  -1.2232479  -2.6985588  -3.9358156  -1.881362    0.47040495 -3.6364372
  -1.4391018  -4.688067   -4.5007124  -2.3305464  -2.700048   -3.1448092
   1.8984761  -0.9046047  -5.1349792  -2.7620718  -4.3072996  -3.4542317
  -1.6708543  -4.8569245  -3.795278   -2.1577492  -6.190428   -7.5620813
  -1.7090299   4.729766   -1.2426262   4.4820995  -0.3435124   0.9073438
   5.1237903   1.9946299  -2.3314931   0.28542534  0.38156492  0.6337116
   2.7686687   4.8627033   1.9456141   0.09420753 -1.4319333  -2.8659027
  -0.05854816  4.941081    1.117424   -2.3173277   0.04154375  1.5937238
   1.2309154  -0.666829    0.7087158  -2.2372942  -4.791397    0.47982934
  -5.27069    -6.299323   -5.28861    -6.681534   -6.308189   -3.8571634
  -7.498931   -5.4768267  -5.666262   -5.213038   -3.6285653  -7.0797567

'ROMEOZ3182"1"/><cite style="padding:3px-0;border:none;margin:rewated sile" class="citation web cs1"><a rel="nofollow" class="external text" href="https://xm.wikipedia.org/wiki/Wikipedia:About" title="ꯔꯥꯖꯁ꯭></span></a></li><li stale="taple weber the hills in India, from Wikimoyage</span></a></li><li class="interlanguage-link interwiki-sw mw-list-item"><a href="https://ps.wikipedia.org/wiki/Rajasthan" title="Rajasthan – Brej\tMisitle="Samase of Rajasthan">Pratapgarh</a></li><li><a href="/wiki/Category:HS5A_Ix-inerhivision of Rajasthan">Pantar</a></li><li><a href="/wiki/Category:Churtu_14-0" class="reference"><a href="#cite_ref-78">^</a></b></span> <span class="reference-text"><a href="https://ms.wikipedia.org/wiki/Ra%C4%9Danasthan" title="Radžastan – Horuca if India">Wililations</a></li><li><a href="/wiki/Category:Wage" title="Badai">Navbarwar</a></li><li><a href="/wiki/Category:Use_Indian_English _Aur dress astardia href="/wiki/Dalibangan" title="Kaman">Ramananda</a></li><li><a href="/

In [ ]:
generate_text(model = model_from_ckpt, start_string="Rajasthan", no_of_chars_to_gen=1000)

Streaming output truncated to the last 5000 lines.
  -7.95179939e+00 -9.36666489e+00 -3.22125173e+00 -5.76201820e+00
  -6.09357882e+00 -7.49338007e+00 -2.83942413e+00 -4.95292044e+00
  -4.40473509e+00 -5.74756098e+00 -5.90959978e+00 -6.55233717e+00
  -4.50286341e+00 -1.10142365e+01 -6.52250481e+00 -5.20294094e+00
  -4.39470911e+00 -6.75884056e+00 -4.50279951e+00 -7.04505825e+00
  -8.34021664e+00 -8.38237858e+00 -6.69593191e+00 -6.64668322e+00
  -4.67738485e+00 -4.62006521e+00 -1.01285229e+01 -8.45459843e+00
  -7.01560211e+00 -6.87910748e+00 -8.12794209e+00 -8.01697254e+00
  -7.31816435e+00 -4.24699593e+00 -6.59554386e+00 -7.40696907e+00
  -4.97427797e+00 -5.38898993e+00 -6.83178234e+00 -7.95251083e+00
  -1.04876509e+01 -1.17959957e+01 -6.55408859e+00 -7.62376547e+00
  -1.09951506e+01 -7.43125916e+00 -5.16775465e+00 -9.95748425e+00
  -5.27745724e+00 -6.29431057e+00 -5.15428209e+00 -9.03745842e+00
  -9.25547886e+00 -6.98853540e+00 -9.21833515e+00 -3.99955511e+00
  -8.64979839e+00 -6.6872

'Rajasthan"><abbr title="Khathis drishan class="mw-cite-backlink"><b><a href="/wiki/Category:States_and_union_territories_of_India" title="Punjab, India">Punjab</a></li><li><a href="/wiki/Mahana" title="Rawatbhat","wgNLef="mw-data:TemplateStyles:r1067148974"/><cite class="citation web cs1"><a rel="nofollow" class="external text" href="https://date.naveduplicated-inline-style" href="mw-data:TemplateStyles:r106564897.<tatlish" title="Bata In="/wiki/Wikipedia:Contant" title="Chittorgarh">Cutlisters</a></th><td class="navbox-list-with-group navbox-list navbox-odd" style="width:100%;padding:0"><div style="handyp"><span>Ittleash in Rajasthan">districts</a></th><td class="navbox-list-with-group navbox-list navbox-even" style="width:100%;padding:0"><div><a href="/wiki/Category:Wikipedia_articles_written_in_Indian_registration_in_Rajasthan" title="Rajasthan – Gower eagle">sportade doth India</a></li></ul></div>\r\n</th>\r\n<td><div id="p-logo" srcset="//upload.wikimedia.org/wikipedia/commons/th

In [ ]:
generate_text(model = model_from_ckpt, start_string="<span>", no_of_chars_to_gen=1000)

NameError: ignored

In [ ]:
for _ in range(15):
  predictions = [[1.,2.,3.,4.]]
  samples = tf.random.categorical(predictions, 1)[-1,0]
  print(samples.numpy())

3
3
1
3
3
2
3
3
2
0
2
3
3
3
3


in the above numbers are varying between 0,1,2 and 3

now when we introduce a very high number the prediction is always the index of highest number

In [ ]:
for _ in range(15):
  predictions = [[1.,2.,3.,4.,55.]]
  samples = tf.random.categorical(predictions, 1)[-1,0]
  print(samples.numpy())

4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


when we introduce 56 which is nearby 55, the prediction vary between 4 and 5 but we will never get 0,1,2,3

In [ ]:
for _ in range(15):
  predictions = [[1.,2.,3.,4.,55.,56.]]
  samples = tf.random.categorical(predictions, 1)[-1,0]
  print(samples.numpy())

5
5
5
4
4
4
5
5
5
4
4
5
5
5
4


let's introduce 100 and 101 then we will get index between 6 and 7

In [ ]:
for _ in range(15):
  predictions = [[1.,2.,3.,4.,55.,56.,100.,101., 1001]]
  samples = tf.random.categorical(predictions, 1)[-1,0]
  print(samples.numpy())

8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
